**CLASSIFICATION MODEL**


In [ ]:
#Importing the useful libraries

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
np.random.seed(42)

In [ ]:
from google.colab import files # Importing the data 
uploaded = files.upload()


Saving governor_state_forecast (1).csv to governor_state_forecast (1).csv


In [ ]:
import pandas as pd ## Read the data 
df = pd.read_csv("governor_state_forecast (1).csv")

In [ ]:
df.head()

,forecastdate,state,district,special,candidate,party,incumbent,model,win_probability,voteshare,p10_voteshare,p90_voteshare
0,2018-10-11,AK,NaN,NaN,Mike Dunleavy,R,False,classic,0.7413,39.55,32.10,46.93
1,2018-10-11,AK,NaN,NaN,Mark Begich,D,False,classic,0.2211,31.28,22.43,40.15
2,2018-10-11,AK,NaN,NaN,Bill Walker,U,True,classic,0.0376,25.74,19.22,32.54
3,2018-10-11,AK,NaN,NaN,Others,NaN,False,classic,0.0000,3.42,1.12,6.26
4,2018-10-11,AL,NaN,NaN,Kay Ivey,R,True,classic,0.9866,58.59,53.95,63.21


**Finding Missing Values**

In [ ]:
df.isnull().any() ## finding null values 

forecastdate       False
state              False
district            True
special             True
candidate          False
party               True
incumbent          False
model              False
win_probability    False
voteshare          False
p10_voteshare      False
p90_voteshare      False
dtype: bool

**Defining The Datatype**

In [ ]:
df.dtypes ## finding data type

forecastdate        object
state               object
district           float64
special            float64
candidate           object
party               object
incumbent             bool
model               object
win_probability    float64
voteshare          float64
p10_voteshare      float64
p90_voteshare      float64
dtype: object

**Preparing the data for Classification model**

In [ ]:
data = df.drop(['forecastdate',	'state', 'district', 'special', 'candidate', 'party', 'model'	], axis=1) ## removing unnessery columns 

In [ ]:
data['incumbents']= pd.factorize(data.incumbent)[0]  ## converting catagorical values in to numaric format 

In [ ]:
df1 = data.drop('incumbent', axis= 1)  ## droping major category 



In [ ]:
df1

##feature lable is  [win_probability,	voteshare,	p10_voteshare,	p90_voteshare]
## target lable is  [incumbents]

,win_probability,voteshare,p10_voteshare,p90_voteshare,incumbents
0,0.7413,39.55,32.10,46.93,0
1,0.2211,31.28,22.43,40.15,0
2,0.0376,25.74,19.22,32.54,1
3,0.0000,3.42,1.12,6.26,0
4,0.9866,58.59,53.95,63.21,1
...,...,...,...,...,...
7738,0.2974,46.45,41.43,51.40,1
7739,0.0000,3.27,1.16,5.85,0
7740,0.9974,64.62,57.65,71.59,0
7741,0.0026,31.57,24.56,38.53,0


**RandomForestClassifier**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
###make the data
x = df1.drop('incumbents', axis=1) ## feature variable 
y = df1['incumbents']  ## traget variable

## splitting the data
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2)

## instantiate Random farest classifier.
clf =RandomForestClassifier()

## fitting the model to the data
clf.fit(x_train, y_train)

###  model evaluation
clf.score(x_test,y_test)

0.9845061329890252

In [ ]:
clf.predict(x_test) ## predection 

array([0, 0, 1, ..., 0, 0, 0])

**compare predection to our truth lable**

In [ ]:
y_pred = clf.predict(x_test)  ### the mean value of predicted value and actual value is shown bellow
np.mean(y_pred == y_test)

0.9845061329890252

**Classification model evalution matric**



In [ ]:
## importing library cross val score from sklearn. 

from sklearn.model_selection import cross_val_score   
clf=RandomForestClassifier(n_estimators=100)  ## hear n estimaters is 100.

cross_val_score = cross_val_score(clf,x,y,cv = 5)
cross_val_score


array([0.96578438, 0.97224015, 0.98515171, 0.95348837, 0.94056848])

 ROC Curve

In [ ]:
from sklearn.metrics import roc_curve   ## importing roc curve library 
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2) ## training is 80% testing is 20%
 
clf.fit(x_train,y_train) ## fitting the model 

##make predection with probability 
y_probs = clf.predict_proba(x_test)

y_probs[:8]


array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [ ]:
y_probs_positive = y_probs[:, 1] ## y probs positive 
y_probs_positive[:10]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
y_preds = clf.predict(x_test)  ## y preds score 

confusion_matrix(y_test, y_preds) ## confusion matric


array([[1294,    4],
       [  11,  240]])

**visualizing Confusion matrix with cross tab**

In [ ]:
pd.crosstab(y_test, y_preds, rownames=['Actual lable'],  ## visualizing values in cross table 
            colnames = ['predicted lables'])

predicted lables,0,1
Actual lable,,
0,1294,4
1,11,240


**Classification Report**

In [ ]:
from sklearn.metrics import classification_report   ## importing the library 
report = classification_report(y_test, y_preds) ## clasification report 
print(report)

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1298
           1       0.98      0.96      0.97       251

    accuracy                           0.99      1549
   macro avg       0.99      0.98      0.98      1549
weighted avg       0.99      0.99      0.99      1549

